Premium vs Freemium

Find the total number of downloads for paying and non-paying users by date. Include only records where non-paying customers have more downloads than paying customers. The output should be sorted by earliest date first and contain 3 columns date, non-paying downloads, paying downloads.

In [1]:
import pandas as pd
import numpy as np

In [23]:
ms_user_dimension = pd.read_excel('../CSV/ms_user_dimension.xlsx', header=1)
ms_user_dimension.head()

,user_id,acc_id
0,1,716
1,2,749
2,3,713
3,4,744
4,5,726


In [24]:
ms_acc_dimension = pd.read_csv("../CSV/ms_acc_dimension.csv")
ms_acc_dimension = ms_acc_dimension.iloc[:, :2]
ms_acc_dimension.head()

,acc_id,paying_customer
0,700,no
1,701,no
2,702,no
3,703,no
4,704,no


In [25]:
ms_download_facts = pd.read_excel("../CSV/ms_download_facts.xlsx", skiprows=1)
ms_download_facts.head()

,date,user_id,downloads
0,2020-08-24,1,6
1,2020-08-22,2,6
2,2020-08-18,3,2
3,2020-08-24,4,4
4,2020-08-19,5,7


In [26]:
df=pd.merge(ms_user_dimension, ms_acc_dimension, how = 'inner',left_on = ['acc_id'], right_on=['acc_id'])
df.head()

,user_id,acc_id,paying_customer
0,1,716,no
1,2,749,yes
2,3,713,no
3,4,744,yes
4,5,726,yes


In [27]:
df1= pd.merge(df, ms_download_facts, how = 'inner',left_on = ['user_id'], right_on=['user_id'])
df1.head()

,user_id,acc_id,paying_customer,date,downloads
0,1,716,no,2020-08-24,6
1,2,749,yes,2020-08-22,6
2,3,713,no,2020-08-18,2
3,4,744,yes,2020-08-24,4
4,5,726,yes,2020-08-19,7


In [28]:
x=df1.pivot_table(index=['date'],columns=['paying_customer'],values=['downloads'],aggfunc='sum')
x

downloads    
paying_customer        no yes
date                         
2020-08-15             11  19
2020-08-16             15  14
2020-08-17             45   9
2020-08-18             10   7
2020-08-19             13  13
2020-08-20             13  28
2020-08-21             32  17
2020-08-22             15  48
2020-08-23             12  23
2020-08-24              6  39
2020-08-25             23  30

In [29]:
df2=pd.DataFrame(x.to_records())
df2

,date,"('downloads', 'no')","('downloads', 'yes')"
0,2020-08-15,11,19
1,2020-08-16,15,14
2,2020-08-17,45,9
3,2020-08-18,10,7
4,2020-08-19,13,13
5,2020-08-20,13,28
6,2020-08-21,32,17
7,2020-08-22,15,48
8,2020-08-23,12,23
9,2020-08-24,6,39


In [30]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   date                  11 non-null     object
 1   ('downloads', 'no')   11 non-null     int64 
 2   ('downloads', 'yes')  11 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 396.0+ bytes


In [31]:
y = x.to_records()
y

rec.array([('2020-08-15', 11, 19), ('2020-08-16', 15, 14),
           ('2020-08-17', 45,  9), ('2020-08-18', 10,  7),
           ('2020-08-19', 13, 13), ('2020-08-20', 13, 28),
           ('2020-08-21', 32, 17), ('2020-08-22', 15, 48),
           ('2020-08-23', 12, 23), ('2020-08-24',  6, 39),
           ('2020-08-25', 23, 30)],
          dtype=[('date', 'O'), ("('downloads', 'no')", '<i8'), ("('downloads', 'yes')", '<i8')])

In [33]:
df2.columns = df2.columns.str.replace("[()]","").str.replace("[' ']","").str.replace("[,]","").str.replace("downloads","")
df2

,date,"('', 'no')","('', 'yes')"
0,2020-08-15,11,19
1,2020-08-16,15,14
2,2020-08-17,45,9
3,2020-08-18,10,7
4,2020-08-19,13,13
5,2020-08-20,13,28
6,2020-08-21,32,17
7,2020-08-22,15,48
8,2020-08-23,12,23
9,2020-08-24,6,39


In [34]:
df3=df2.fillna(0)
df3

,date,"('', 'no')","('', 'yes')"
0,2020-08-15,11,19
1,2020-08-16,15,14
2,2020-08-17,45,9
3,2020-08-18,10,7
4,2020-08-19,13,13
5,2020-08-20,13,28
6,2020-08-21,32,17
7,2020-08-22,15,48
8,2020-08-23,12,23
9,2020-08-24,6,39


In [35]:
df3['diff'] = df3["('', 'no')"]-df3["('', 'yes')"]
df3

,date,"('', 'no')","('', 'yes')",diff
0,2020-08-15,11,19,-8
1,2020-08-16,15,14,1
2,2020-08-17,45,9,36
3,2020-08-18,10,7,3
4,2020-08-19,13,13,0
5,2020-08-20,13,28,-15
6,2020-08-21,32,17,15
7,2020-08-22,15,48,-33
8,2020-08-23,12,23,-11
9,2020-08-24,6,39,-33


In [36]:
df4 = df3[df3["diff"] > 0]
df4

,date,"('', 'no')","('', 'yes')",diff
1,2020-08-16,15,14,1
2,2020-08-17,45,9,36
3,2020-08-18,10,7,3
6,2020-08-21,32,17,15


In [38]:
df4.rename(columns={"('', 'no')" : 'no', "('', 'yes')":'yes'}, inplace=True)
df4

/var/folders/8j/skd2y7rd5k752cwx0v97h73c0000gn/T/ipykernel_74416/3331407537.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4.rename(columns={"('', 'no')" : 'no', "('', 'yes')":'yes'}, inplace=True)


,date,no,yes,diff
1,2020-08-16,15,14,1
2,2020-08-17,45,9,36
3,2020-08-18,10,7,3
6,2020-08-21,32,17,15


In [39]:
result = df4[["date","no","yes"]].sort_values("date")
result

,date,no,yes
1,2020-08-16,15,14
2,2020-08-17,45,9
3,2020-08-18,10,7
6,2020-08-21,32,17


Solution Walkthrough
This solution walkthrough explains the code snippet provided and the problem it solves. The code utilizes the pandas library to merge dataframes and perform data manipulation tasks.

Understanding The Data
The code snippet uses three dataframes - ms_user_dimension, ms_acc_dimension, and ms_download_facts.

ms_user_dimension contains information about users, including their user_id.
ms_acc_dimension contains information about user accounts, including the acc_id.
ms_download_facts contains the number of downloads for each user, identified by user_id.
The Problem Statement
The goal is to find the total number of downloads for paying and non-paying users by date. Only records where non-paying customers have more downloads than paying customers should be included. The output should be sorted by the earliest date first and contain three columns: date, non-paying downloads, and paying downloads.

Breaking Down The Code
Let's break down the code snippet step-by-step:

Import required libraries: The code begins by importing the pandas library as pd and the numpy library as np.

Merge dataframes: The first merge operation is performed using the pd.merge() function. It merges ms_user_dimension and ms_acc_dimension dataframes based on the 'acc_id' column using an inner join. The resulting dataframe is assigned to the variable df.

Perform second merge operation: The second merge operation is performed using the pd.merge() function. It merges the df dataframe from the previous step and ms_download_facts dataframe based on the 'user_id' column using an inner join. The resulting dataframe is assigned to the variable df1.

Create a pivot table: The x variable is assigned the pivot table created using the pivot_table() function. It takes df1 as input and performs the following operations:

Index: 'date' column
Columns: 'paying_customer' column
Values: 'downloads' column
Aggregation function: 'sum'
Convert to a dataframe: The x pivot table is converted to a dataframe using the to_records() method. The resulting dataframe is assigned to df2.

Clean column names: The column names in df2 are cleaned and modified using the str.replace() method to remove unwanted characters.

Fill NaN values: The df3 dataframe is created by filling any NaN values in df2 with 0 using the fillna(0) method.

Calculate difference: A new column 'diff' is added to the df3 dataframe that represents the difference between the 'no' and 'yes' columns.

Filter records: The df4 dataframe is created by selecting only the rows where the 'diff' column is greater than 0.

Sort and select columns: The final result is obtained by selecting the 'date', 'no', and 'yes' columns from the df4 dataframe and sorting the result by the 'date' column.

Bringing It All Together
The code snippet performs data merging, pivoting, cleaning, filtering, and sorting to achieve the desired output. It uses pandas functions and methods, such as merge(), pivot_table(), fillna(), str.replace(), and sort_values().

Conclusion
The provided code snippet merges multiple dataframes, calculates the number of downloads for paying and non-paying users by date, and filters the records where non-paying customers have more downloads than paying customers. The resulting output is a dataframe containing the earliest date first, along with the number of downloads for non-paying and paying users.